In [1]:
import sys
sys.path.append("./Bspli/")
import os  
import faiss
import time
import numpy as np 
import index 
import torch
import pandas as pd

mnist = np.load("dataset/mnist-784-euclidean.npy")
print(f'mnist data shape: {mnist.shape}')
print(f'mnist dtype: {mnist.dtype}')

# mnist = np.random.rand(60000, 2)
# mnist = mnist.astype(np.float32)
# print(f'mnist dtype: {mnist.dtype}')
# print(f'mnist data shape: {mnist.shape}')


mnist data shape: (60000, 784)
mnist dtype: float32


In [2]:
%%time
# Brute Search
flat = faiss.IndexFlatL2(mnist.shape[1])
flat.add(mnist)

D, FLAT_I = flat.search(mnist[0].reshape(1, mnist.shape[1]), k=100) 
print(f'brute query: {FLAT_I}')

brute query: [[    0 32248  8728 18932 30483 24149 42338 52295 26251 50173 53634 24330
  54159 57528  1482 53428 18123 31379 52864 10536 29719 36087 30489 23947
  20034 52057 33825 21654 31008 55208 22477 44282 47968 54203 19825  1634
  27378 33909 15378 24708 34474 26413 16017 46824 46358  1516 34557 16832
  21629 29021 10740 24107  5688 52665  1864  5036 39031  1978 40546 22322
  52231 37284 24730  5970 21976 16945  9568 36697 25675 54189 11396 42555
  33445 52540 44263 18404 19186 24232 54184 25762 14736 33970  5210 59212
   8642 22569 15052  2933  6772 22963  6516   832 21244 21583 35838 59846
  21210 13502 52559 13862]]
CPU times: total: 93.8 ms
Wall time: 98.7 ms


In [3]:
mnist_tensor = torch.from_numpy(mnist)
print(f'mnist tensor shape: {mnist_tensor.shape}')

idx = index.Indexing(
    gl_size=80000, 
    ll_size=1000,
    g_epoch_num=5,
    l_epoch_num=30,
    g_hidden_size=5,
    l_hidden_size=10,
    l_block_range=25,
    random_partitioning=False
)
idx.train(mnist_tensor)

print(f"local model len:{len(idx._l_model)}")

mnist tensor shape: torch.Size([60000, 784])
torch.Size([60000, 785])
first stage partitioning finish
partitioning blocks : 1
training local model


d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:988: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)


1, 100 loss: 0.2698622512817383
1, 200 loss: 0.2705184745788574
1, 300 loss: 0.2413215446472168
1, 400 loss: 0.24074575424194336
1, 500 loss: 0.19300430297851562
1, 600 loss: 0.19833263397216797
2, 100 loss: 0.17897863388061525
2, 200 loss: 0.17321250915527345
2, 300 loss: 0.17073883056640626
2, 400 loss: 0.15729809761047364
2, 500 loss: 0.15273608207702638
2, 600 loss: 0.15807568550109863
3, 100 loss: 0.1550144386291504
3, 200 loss: 0.13744481086730956
3, 300 loss: 0.15629261016845702
3, 400 loss: 0.15460164070129395
3, 500 loss: 0.15858479499816894
3, 600 loss: 0.15264425277709961
4, 100 loss: 0.14636314392089844
4, 200 loss: 0.1532992935180664
4, 300 loss: 0.15706639289855956
4, 400 loss: 0.16868106842041017
4, 500 loss: 0.1564497947692871
4, 600 loss: 0.15770397186279297
5, 100 loss: 0.13963208198547364
5, 200 loss: 0.15714559555053711
5, 300 loss: 0.14544346809387207
5, 400 loss: 0.14833748817443848
5, 500 loss: 0.1663178825378418
5, 600 loss: 0.15412880897521972
6, 100 loss: 0.14

d:\study\bspli\./Bspli\utils\Partitioning.py:113: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ma = torch.tensor(means)
d:\conda\envs\ml\lib\site-packages\torch\nn\modules\loss.py:928: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


In [4]:
%%time

def recall(pred, true):
    x = np.isin(pred, true)
    return x.sum() / true.size

qp = torch.from_numpy(mnist[0])
# print(qp)
pred = idx.query(qp, k=100)
print(f"recall: {recall(pred, FLAT_I)}")

pred = pred.to(torch.int)
print(f"pred: {pred}")

recall: 0.48
pred: tensor([    0,  8728, 24149, 42338, 52295, 50173, 53634, 24330,  1482, 18123,
        23947, 20034, 52057, 44282, 54203, 19825, 33909, 24708, 46824,  1516,
        34557, 29021, 10740, 24107, 52665,  1864,  5036, 40546, 22322, 21976,
        36697, 25675, 54189, 11396, 42555, 52540, 44263, 25762, 14736,  5210,
        59212, 22569, 21244, 35838, 21210, 13502, 52559, 13862, 41980, 43997,
        53812, 15975, 50187, 28384, 24271, 21661, 15907,  4622, 50301, 38896,
        15444, 50071, 45567, 26313,  2395, 11936, 16112, 49334, 44476, 18189,
        17298, 26364, 36242, 18197, 25557,  8230, 29649, 21543, 45432,  3005,
        14543, 15415, 40432, 22542, 33607, 43067,  9652, 21563, 25623, 54283,
        52310, 42971, 24281, 20570, 54131,  2037, 25720, 36695, 32001,  6274],
       dtype=torch.int32)
CPU times: total: 359 ms
Wall time: 56.9 ms


In [5]:
result = []

def benchmark_knn_query(data, index, size=1000, k=100):
    indices = np.random.choice(data.shape[0], size, replace=False)
    query_time = 0
    cur_recall = 0

    # query
    for i in indices:
        q = torch.from_numpy(data[i])
        start = time.time()
        qk = index.query(q, k=100)
        query_time += (time.time() - start)
        D, FLAT_I = flat.search(data[i].reshape(1, data.shape[1]), k=k) 
        cur_recall += recall(qk, FLAT_I)
    result.append((query_time/1000, cur_recall/1000))

In [6]:
# recall(pred, FLAT_I)

benchmark_knn_query(mnist, idx, size=1000, k=100)

In [7]:
print(result)

[(0.047600539445877076, 0.42035999999999946)]
